In [1]:
import pandas as pd
import multiprocessing
pd.set_option('display.max_colwidth', None)
import re
import os
os.chdir(r'D:\Хакатон')
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel("ИКРБС_2020-2024_гг..xlsx")

In [3]:
df.fillna('Отсутствует', inplace=True)

In [4]:
df.drop(["№ п/п", "Регистрационный номер"],axis=1,inplace=True)

In [5]:
df['text_data'] = df['Наименование отчёта'] + ' ' + df['Реферативное описание отчета'] + ' ' + df['Ключевые слова']



In [6]:
df = df.sample(frac=1)


In [12]:
df

Наименование отчёта  \
26398                                                                                                                                                                       Представления нередуктивных групп и геометрия многообразий   
64037                                                                                                            РОЛЬ ЭНДОТЕЛИЯ В РЕГУЛЯЦИИ СОЗРЕВАНИЯ ООЦИТА ПРИ ЛЕЧЕНИИ БЕСПЛОДИЯ МЕТОДАМИ ВСПОМОГАТЕЛЬНЫХ РЕПРОДУКТИВНЫХ ТЕХНОЛОГИЙ   
12645                                        Создание Лаборатории мозаики аутоиммунитета с целью изучения причин и механизмов аутоимунных болезней и использования иммунологических инструментов в их диагностике и лечении (итоговый)   
41256                                                                                                                                              Развитие методов космической геодезии для изучения современных движений земной коры   
52412                  Разработка и тестирование прототипа цифрового ассистента для обучения русскому языку иностранных студентов в системе высшего образования (договор № 163ГС1ИИС12-D7/80640 от 17 ноября 2022 г.) (заключительный)   
...                                                                                                                                                                                                                                ...   
74979                                                                             Разработка прототипов серверного и мобильного приложений для групповой видеосвязи с использованием нейронных сетей для распознавания и перевода речи   
55520                                                                                                    Теория представлений W—алгебр, аффинных и тороидальных квантовых групп и ее приложения к геометрии, топологии и комбинаторике   
57438                                                                                        Создать новые высокопродуктивные сорта зерновых колосовых культур, устойчивые к стресс факторам на основе комплексного изучения генофонда   
56937  Спин-орбитальное и обменное взаимодействие как движущая сила формирования электронной и спиновой структуры новых материалов/топологических наносистем для использования в наноэлектронике и спинтронике (промежуточный, этап 9)   
29188          ИССЛЕДОВАНИЕ МОЛЕКУЛЯРНЫХ И КЛЕТОЧНЫХ МЕХАНИЗМОВ ТРОМБОТИЧЕСКИХ ОСЛОЖНЕНИЙ И НАРУШЕНИЙ СИСТЕМЫ КОМПЛЕМЕНТА ПРИ КОРОНАВИРУСНОЙ ИНФЕКЦИИ (COVID-19). ПОИСК ПРОГНОСТИЧЕСКИХ ПРИЗНАКОВ ДЛЯ ОЦЕНКИ ТЯЖЕСТИ ТЕЧЕНИЯ COVID-19.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [7]:
sentence_example = df.iloc[1]["text_data"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


In [8]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: РОЛЬ ЭНДОТЕЛИЯ В РЕГУЛЯЦИИ СОЗРЕВАНИЯ ООЦИТА ПРИ ЛЕЧЕНИИ БЕСПЛОДИЯ МЕТОДАМИ ВСПОМОГАТЕЛЬНЫХ РЕПРОДУКТИВНЫХ ТЕХНОЛОГИЙ Цель исследования: изучение роли эндотелия в регуляции функционального состояния микроокружения ооцита и его созревания при лечении бесплодия методами ВРТ.  Задачи исследования: 1.Оценка содержания факторов эндотелиальной регуляции сосудистого тонуса в крови, фолликулярной жидкости и клетках кумулюса у женщин с оптимальным и неудовлетворительным результатами стимуляции яичников гонадотропинами при лечении бесплодия методами ВРТ. 2. Исследование концентраций тромбогенных и антитромбогенных факторов эндотелия в крови у женщин с оптимальным и неудовлетворительным результатами стимуляции яичников гонадотропинами при лечении бесплодия методами ВРТ. 3. Оценка содержания факторов, регулирующих противовоспалительную и пролиферативную функции эндотелия, в крови и фолликулярной жидкости у женщин с оптимальным и неудовлетворительным результатами стимуляции яичников

In [9]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [10]:
tokenize_sentence(sentence_example)

['рол',
 'эндотел',
 'в',
 'регуляц',
 'созреван',
 'ооцит',
 'при',
 'лечен',
 'бесплод',
 'метод',
 'вспомогательн',
 'репродуктивн',
 'технолог',
 'цел',
 'исследован',
 'изучен',
 'рол',
 'эндотел',
 'регуляц',
 'функциональн',
 'состоян',
 'микроокружен',
 'ооцит',
 'созреван',
 'лечен',
 'бесплод',
 'метод',
 'врт',
 'задач',
 'исследован',
 '1.оценк',
 'содержан',
 'фактор',
 'эндотелиальн',
 'регуляц',
 'сосудист',
 'тонус',
 'кров',
 'фолликулярн',
 'жидкост',
 'клетк',
 'кумулюс',
 'женщин',
 'оптимальн',
 'неудовлетворительн',
 'результат',
 'стимуляц',
 'яичник',
 'гонадотропин',
 'лечен',
 'бесплод',
 'метод',
 'врт',
 '2',
 'исследован',
 'концентрац',
 'тромбоген',
 'антитромбоген',
 'фактор',
 'эндотел',
 'кров',
 'женщин',
 'оптимальн',
 'неудовлетворительн',
 'результат',
 'стимуляц',
 'яичник',
 'гонадотропин',
 'лечен',
 'бесплод',
 'метод',
 'врт',
 '3',
 'оценк',
 'содержан',
 'фактор',
 'регулир',
 'противовоспалительн',
 'пролиферативн',
 'функц',
 'эндотел',
 '

In [11]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['text_data'].values.astype('U'))

MemoryError: Unable to allocate 10.5 GiB for an array with shape (81868,) and data type <U34408

In [ ]:
y = df['ГРНТИ']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)

In [ ]:
model = SGDClassifier(max_iter=5000)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:

accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

In [ ]:
# Новый текст
new_text = "СТРОИТЕЛЬСТВО СТАНКОВ"

# Преобразование нового текста в вектор признаков
new_vector = vectorizer.transform([new_text])

# Получение предсказания рубрики ГРНТИ
predicted_grnti = model.predict(new_vector)

print("Предсказанная рубрика ГРНТИ:", predicted_grnti)